# Observation/GTI handling

### Gammapy Coding Sprint - Madrid 2018

#### David Fidalgo


## 1. Observation handling

### What do we have?
 1. Create _observations_ from a **DataStore** object ("standard" way)
   - super convenient, just *obs_id* needed
   - but *Index files* necessary
   - *Events* and *IRFS* not stored in memory
 2. New in 0.8: create observation from "scratch" 
   - not so convenient
   - no *Index files* needed
   - everything stored in memory
   - still not fully implemented (work for this week)

In [ ]:
# "standard" way
from gammapy.data import DataStore

data_store = DataStore.from_dir("$GAMMAPY_DATA/cta-1dc/index/gps/")
obs_ids = [110380, 111140, 111159]
obs_list = data_store.obs_list(obs_ids)

In [87]:
# from "scratch"
from gammapy.data import ObservationCTA, EventList, GTI
from gammapy.irf import (
    EnergyDependentMultiGaussPSF,
    EffectiveAreaTable2D,
    EnergyDispersion2D,
    Background3D,
)

filename = "$GAMMAPY_DATA/cta-1dc/data/baseline/gps/gps_baseline_110380.fits"
event_list = EventList.read(filename)
gti = GTI.read(filename)

filename = "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
aeff = EffectiveAreaTable2D.read(filename)
bkg = Background3D.read(filename)
edisp = EnergyDispersion2D.read(filename, hdu="Energy Dispersion")
psf = EnergyDependentMultiGaussPSF.read(filename, hdu="Point Spread Function")

obs = ObservationCTA(
    obs_id=event_list.table.meta["OBS_ID"],
    events=event_list,
    gti=gti,
    psf=psf,
    aeff=aeff,
    edisp=edisp,
    bkg=bkg,
    pointing_radec=event_list.pointing_radec,
    observation_live_time_duration=event_list.observation_live_time_duration,
    observation_dead_time_fraction=event_list.observation_dead_time_fraction,
)

# or create it from existing observation
obs = obs_list[0].to_observation_cta()

## 2. GTI handling

### What do we have?
 - existing GTI class not used
 - new observation class allows for selections in time

In [92]:
from astropy.time import Time

#print(obs.events.time.min().utc.datetime, obs.events.time.max().utc.datetime)
time_interval = Time(['2021-01-21T12:00:00', '2021-01-21T12:10:00'])
obs.events = obs.events.select_time(time_interval)  # this does not work with DataStoreObservation

obs_time = (time_interval[1] - time_interval[0]).to('s')
obs.observation_live_time_duration = obs_time * (1 - obs.observation_dead_time_fraction)